In [ ]:
import sys
import os

import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt
from PIL import Image

from keras.models import Model, Sequential
from keras.layers import Dense, Reshape, UpSampling2D, Conv2D, BatchNormalization, \
Activation, Input, LeakyReLU, Dropout, ZeroPadding2D, Flatten
from keras.optimizers import Adam

In [ ]:
!unzip data.zip

In [ ]:
!ls data | head

In [ ]:
imgs = os.listdir('data')

In [ ]:
n_instances = len(imgs)

In [ ]:
n_instances

In [ ]:
index = np.random.randint(low=0, high=n_instances)
img_path = os.path.join('data', imgs[index])

In [ ]:
img = Image.open(img_path)

In [ ]:
img

In [ ]:
img_rows, img_cols = img.size

In [ ]:
X_train = np.zeros((n_instances, img_rows, img_cols, 3))

In [ ]:
for i, img_name in tqdm(enumerate(imgs), total=n_instances):
  img_path = os.path.join('data', img_name)
  pil_image = Image.open(img_path)
  X_train[i] = np.array(pil_image)

In [ ]:
X_train.shape

In [ ]:
class DCGAN():
  def __init__(self, img_rows, img_cols, channels, latent_dim):
    self.img_rows = img_rows
    self.img_cols= img_cols
    self.channels = channels
    self.img_shape = (img_rows, img_cols, channels)

    self.latent_dim = latent_dim

    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

    self.discriminator.trainable = False
    self.generator = self.build_generator()

    input = Input(shape=(self.latent_dim,))
    generated_img = self.generator(input)
    is_valid_img = self.discriminator(generated_img)

    self.combined = Model(input, is_valid_img)
    self.combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

  def build_generator(self, show_model_summary=True):
    model = Sequential()
    model.add(Dense(128 * 16 * 16, activation='relu', input_dim=self.latent_dim))
    model.add(Reshape((16, 16, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))

    if show_model_summary:
      print('Generator summary: ')
      model.summary()

    input = Input(shape=(self.latent_dim,))
    output = model(input)

    return Model(input, output)

  def build_discriminator(self, show_model_summary=True):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, padding='same', input_shape=self.img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    if show_model_summary:
      print('Discriminator summary:')
      model.summary()

    input = Input(shape=self.img_shape)
    output = model(input)

    return Model(input, output)

  def train(self, X_train, epochs, batch_size=128, save_interval=50):
    X_train = X_train / 127.5 - 1

    y_real = np.ones((batch_size,))
    y_fake = np.zeros((batch_size,))

    g_losses = []
    d_losses = []
    d_accs = []

    for epoch in tqdm(range(epochs)):
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      real_imgs_input = X_train[idx]

      noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
      fake_imgs_input = self.generator.predict(noise)

      d_loss_real = self.discriminator.train_on_batch(real_imgs_input, y_real)
      d_loss_fake = self.discriminator.train_on_batch(fake_imgs_input, y_fake)

      d_loss = np.add(d_loss_real, d_loss_fake) * 0.5
      d_losses.append(d_loss[0])
      d_accs.append(d_loss[1])

      g_loss = self.combined.train_on_batch(noise, y_real)
      
      g_losses.append(g_loss)

      if epoch % save_interval == 0:
        self.save_images(epoch)

    return g_losses, d_losses, d_accs
  
  def generate_image(self, input):
    gen_img = self.generator.predict(input)
    gen_img = gen_img * 0.5 + 0.5
    return gen_img
  
  def save_images(self, epoch):
    r, c = 5, 5

    noise = np.random.normal(0, 1, (r * c, self.latent_dim))
    gen_imgs = self.generate_image(noise)

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
      for j in range(c):
        axs[i, j].imshow(gen_imgs[cnt])
        axs[i, j].axis('off')
        cnt += 1
    fig.savefig('generated/anime_%d.png' % epoch)
    plt.close()

In [ ]:
channels = 3
latent_dim = 100
dcgan = DCGAN(img_rows, img_cols, channels, latent_dim)

In [ ]:
!mkdir generated

In [ ]:
epochs = 5001
batch_size = 32
save_interval = 50

g_losses, d_losses, d_accs = dcgan.train(X_train, epochs, batch_size, save_interval)

In [ ]:
generated_imgs = os.listdir('generated')

In [ ]:
#Image.open(os.path.join('generated', generated_imgs[np.random.randint(0, len(generated_imgs))]))
Image.open(os.path.join('generated', generated_imgs[len(generated_imgs) - 1]))

In [ ]:
def show_loss(g_losses, d_losses):
  plt.figure(figsize=(20, 10))
  plt.plot(range(epochs), g_losses, label='Generator loss')
  plt.plot(range(epochs), d_losses, label='Discriminator loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend(loc='best')

def show_accuracy(d_accs):
  plt.figure(figsize=(20, 10))
  plt.plot(range(epochs), d_accs, label='Discriminator accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend(loc='best')

In [ ]:
show_loss(g_losses, d_losses)

In [ ]:
show_accuracy(d_accs)